# Multilingual optimization

LLMs are powerful tools which are able to natively take input in one language, and output the result in another. However, for certain languages, the number of characters represented by a single token differs from English. This means certain languages can be much slower for specific models. This varies from model to model, and language to language, so this is only an example of one such instance.

By generating the output in English, and using another service to translate the text, a significant time saving can be achieved.

Note that the accuracy of the translation needs to be tested, and this can be more difficult with streaming use cases.

In [2]:
import sys
!{sys.executable} -m pip install azure-ai-translation-document==1.0.0
!{sys.executable} -m pip install azure-ai-translation-text==1.0.0b1
!{sys.executable} -m pip install azure-ai-textanalytics==5.2.0

print("Successfully installed the required packages")

In [3]:
import os
import time
from openai import AzureOpenAI

from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.document import DocumentTranslationClient
from azure.ai.translation.text import TextTranslationClient, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.exceptions import HttpResponseError

from dotenv import load_dotenv


In [4]:
## Load the enviornment variables
load_dotenv()

## Create the AOAI client and Azure AI Language API client

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-15-preview"
)

credential = TranslatorCredential(os.getenv("translator_api_key"), os.getenv("translator_api_region"))
text_translator = TextTranslationClient(endpoint=os.getenv("translator_api_endpoint"), credential=credential)

ta_credential = AzureKeyCredential(os.getenv("language_api_key"))
text_analytics_client = TextAnalyticsClient(endpoint=os.getenv("language_api_endpoint"), credential=ta_credential)

### A: Base case - using the LLM to translate between languages

**Time taken: 53 seconds**

With LLM generating response in user's query language

In [15]:
message_text = [{"role" : "system", "content" : "You are a helpful AI assistant that helps people find information. Always reply in the user's language. Ex - English for English, Hinglish for Hinglish, Hindi for Hindi. Always reply in 5 bullet points."}, {"role" : "user", "content" : "रोजाना एक्सरसाइज करने के क्या फायदे हैं"}]
start = time.time()

completion = client.chat.completions.create(
  model="gpt-4", # model = "deployment_name"
  messages = message_text,
  temperature=0,
  max_tokens=3000,
  top_p=0.1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)
end = time.time()
total_time = end - start
print("Total time taken : ", total_time)

print(completion.choices[0].message.content)



Total time taken :  53.30297660827637
1. स्वास्थ्य वर्धन: नियमित व्यायाम से हृदय और फेफड़ों की कार्यक्षमता बढ़ती है, और यह ब्लड प्रेशर और कोलेस्ट्रॉल को नियंत्रित करता है।

2. वजन नियंत्रण: एक्सरसाइज करने से कैलोरी बर्न होती है, जिससे वजन नियंत्रण में मदद मिलती है और मोटापे का जोखिम कम होता है।

3. मानसिक स्वास्थ्य: व्यायाम से तनाव, चिंता और अवसाद के लक्षण कम होते हैं और यह मूड को बेहतर बनाता है।

4. ऊर्जा वृद्धि: नियमित शारीरिक गतिविधि से ऊर्जा का स्तर बढ़ता है और थकान कम होती है।

5. दीर्घायु: एक्सरसाइज से जीवन प्रत्याशा बढ़ती है और कई प्रकार के कैंसर, मधुमेह और हृदय रोगों का जोखिम कम होता है।


### B: Using another service to translate the output

**Time taken: 16 seconds**

The LLM is used to generate the output in English. The Azure Translator service is used to convert the output into the user's query language

In [17]:
message_text = [{"role" : "system", "content" : "You are a helpful AI assistant that helps people find information. Always reply in English irrespective of the user language and reply in 5 bullet points"}, {"role" : "user", "content" : "रोजाना एक्सरसाइज करने के क्या फायदे हैं"}]

start = time.time()
completion = client.chat.completions.create(
  model="gpt-4", # model = "deployment_name"
  messages = message_text,
  temperature=0,
  max_tokens=3000,
  top_p=0.1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

## Detect the target language
try:
    documents = [message_text[-1]["content"]]
    response = text_analytics_client.detect_language(documents = documents)[0]
    target_language = response.primary_language.iso6391_name

except Exception as err:
    print("Encountered exception. {}".format(err))

try:

    source_language = "en"
    target_languages = [target_language]
    input_text_elements = [InputTextItem(text = completion.choices[0].message.content)]

    response = text_translator.translate(content = input_text_elements, to = target_languages, from_parameter = source_language)
    translation = response[0]['translations'][0]['text'] if response else None


except HttpResponseError as exception:
    print(f"Error Code: {exception.error.code}")
    print(f"Message: {exception.error.message}")

end = time.time()

total_time = end - start
print("Total time taken : ", total_time)
print(translation)


Total time taken :  16.280856609344482
- **हृदय स्वास्थ्य में सुधार करता है:** नियमित व्यायाम हृदय को मजबूत करता है और परिसंचरण में सुधार करता है, हृदय रोग के जोखिम को कम करता है और रक्तचाप को कम करता है।
- **वजन प्रबंधन:** यह स्वस्थ वजन बनाए रखने, कैलोरी बर्न करने और मेटाबॉलिज्म बढ़ाने में मदद करता है।
- **मानसिक स्वास्थ्य लाभ:** व्यायाम एंडोर्फिन जारी करता है जो अवसाद और चिंता के लक्षणों को कम करने और मूड में सुधार करने में मदद कर सकता है।
- **मांसपेशियों और हड्डियों को मजबूत करता है:** नियमित शारीरिक गतिविधि मांसपेशियों और हड्डियों को मजबूत करती है, ऑस्टियोपोरोसिस के जोखिम को कम करती है और समग्र शरीर की ताकत और संतुलन में सुधार करती है।
- **प्रतिरक्षा प्रणाली को बढ़ावा देता है:** दैनिक व्यायाम में संलग्न होने से प्रतिरक्षा प्रणाली को बढ़ावा मिल सकता है, जिससे शरीर को संक्रमण और बीमारियों से लड़ने में मदद मिलती है।
